<div style="text-align: right"> 
<a href='https://d7vw40z4bofef.cloudfront.net/static/2.37.07-web19/images/service/isometric/flight.svg'>image source</a> </div>

<center><a href="https://rapids.ai/"><img src="img/rapids.png" alt="rapids_logo" style="width: 1500px;"/></a></center>

# <font color='#8735fb'> Model Design and Training
    
In the last activity, we visualized our processed dataset and gained some confidence that our features were correlated to the column we would like to predict. This correlation gives us further confidence that we should be able to train a model to a reasonable level of accuracy.
    
In this Notebook, we will train several models to achieve that high accuracy, and we will compare them.
    
We will break this down into three different types of comparisons:
    
* Comparing the training time required for similar GPU-accelerated (RAPIDS `cuml` and `xgboost`) and CPU-only models (`xgboost` and `sklearn`)
* Comparing the training time and accuracy across model architectures (Gradient Boosted Trees versus Random Forest)
* Comparing the impact of hyperparameters on a single model architecture (`n_estimators`, `max_depth`, `max_features`)
    
As you will see, the training time for the GPU-accelerated models is in the seconds (pay attention to the `wall time`), so we would encourage you to train the model multiple times with different hyperparameters. Can you train faster and with higher accuracy than the defaults in this Notebook? Have fun with this part, as tweaking and optimizing your model can be one of the most common and most fun parts of data science.
    
After training the models, we will save the output for a later inference activity.

In [1]:
# Import RAPIDS and other data science tools
import numpy, pandas
import cupy, cudf, cuml # RAPIDS
import xgboost
from cuml.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier as RandomForestClassifierCPU
import pickle, time

In [2]:
# Define some global directory/file names that change based on the environment

# Set data directory as configured in the workflow
data_dir = '/home/ec2-user/SageMaker/data/'

# Set the name for the previously processed files
processed_data_file = data_dir + 'airlines_processed_data.csv'
processed_data_file_full = data_dir + 'airlines_processed_data_full.csv'

# Specify the path to the model file
xgb_model_file = data_dir + 'xgb_trained_model.model'
rf_model_file = data_dir + 'rf_trained_model.pkl'

In [3]:
# Define import column headers for the data

# These are all the columns in the training dataset
train_columns = ['enc_Flight_Number_Reporting_Airline', 'dest_long', 'enc_ArrTime', 'enc_Origin', 'Quarter',\
                 'dest_elevation', 'enc_Dest', 'origin_elevation', 'enc_OriginCityName', 'AirTime', 'DestCityMarketID',\
                 'origin_lat', 'Month', 'enc_DepTime', 'DepDelay', 'enc_Airline', 'enc_CRSArrTime', 'DayOfWeek',\
                 'enc_DestCityName', 'enc_CRSDepTime', 'enc_Reporting_Airline', 'origin_long', 'Distance', 'DepDel15',\
                 'dest_lat', 'OriginCityMarketID']

# This is the column the model will predict
predict_column =  ['ArrDel15']

# This is a sorted list of all columns for data visualization and exploration
correlation_columns = sorted(train_columns + predict_column)

In [4]:
# In this case we are training our final model, so we use the larger dataset
# processed_data = cudf.read_csv(processed_data_file)
processed_data = cudf.read_csv(processed_data_file_full)

### <font color='#8735fb'> Splitting the Data

Before we do any training, we will need to split our data set into a `train` dataset and a `test` dataset. If you've ever taken another machine learning course, you have probably heard all about this. A general rule of thumb is to save at least 20 percent of your full dataset (often more) to validate the results of your model. You should never train your model against this dataset. By having a reserved test dataset, we can have confidence that our trained model is generalized and has not overfitted to the training data.
    
In this case, we are splitting the dataset before training, but in a more realistic setting, you may actually want to do this during the data preprocessing step to guarantee that you don't accidentally train on your test data. As a result of this, you would have two data files `train_airlines_processed_data_full.csv` and `test_airlines_processed_data_full.csv`. And you would only ever use the test dataset at the end of the training process. Note that here, we are setting the `random_state` field, which ensures that the same data file will always be split the same way, removing randomness.

In [5]:
# Split the preprocessed data into training data and testing data
train_test_split = cuml.preprocessing.model_selection.train_test_split

X_train, X_test, \
y_train, y_test = train_test_split(X = processed_data[train_columns],  
                                   y = processed_data[predict_column], 
                                   train_size=0.80, random_state=42)
print(f'Full dataset has {processed_data.shape[0]} rows with {processed_data.shape[1]} columns')
print(f'Training dataset has {X_train.shape[0]} rows with {X_train.shape[1]} columns')
print(f'Test dataset has {X_test.shape[0]} rows with {X_test.shape[1]} columns')

Full dataset has 7268232 rows with 31 columns
Training dataset has 5814585 rows with 26 columns
Test dataset has 1453647 rows with 26 columns


In [6]:
# Create Pandas versions of our datasets to benchmark our GPU models against CPU models
X_train_cpu = X_train.to_pandas()
X_test_cpu = X_test.to_pandas()
y_train_cpu = y_train.to_pandas()

In [7]:
# Convert the dataset into a format that XGBoost can easily consume
dtrain = xgboost.DMatrix(data=X_train, label=y_train)
dtest = xgboost.DMatrix( X_test, y_test)

### <font color='#8735fb'> XGBoost
 
XGBoost is a powerful library that is often used as a go-to first model attempt for many machine learning problems. Just look at how many of the top Kaggle submission are leveraging this library!
    
The XGBoost Python module has both [GPU implementations](https://resources.nvidia.com/en-us-xgboost) and CPU implementations built right into it. See the documentation for full details, but the implementation can easily be toggled by setting the `tree_method` parameter to `gpu_hist` for GPU and `hist` for CPU, as seen in this cell.
    
For this model, we will be modifying the number of trees in the ensemble (`n_estimators`), the maximum depth of each tree (`max_depth`), and how many features can be considered at each split (`max_features`).
    
Note: Training may take several minutes to complete. As such, patience can be a key data-scientist skill to master.

In [8]:
# GPU XGBoost parameters
model_params = {
    'n_estimators' : 300,    
    'max_depth'    : 5,
    'max_features' : .5,
    'tree_method'  : 'gpu_hist' }

# CPU XGBoost parameters
model_params_cpu = {
    'n_estimators' : 300,
    'max_depth'    : 5,    
    'max_features' : .5,
    'tree_method'  : 'hist',
    'nthread'      : 32 }

# If the prediction is above this threshold we consider the flight as predicted "late"
threshold = .5

In [9]:
%%time

xgb_trained_model = xgboost.train(dtrain=dtrain, params=model_params,
                              num_boost_round=model_params['n_estimators'], verbose_eval=False)

[22:34:59] WARNING: /opt/conda/envs/rapids/conda-bld/xgboost_1612969655237/work/src/learner.cc:541: 
Parameters: { max_features, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


CPU times: user 2.74 s, sys: 1.76 s, total: 4.5 s
Wall time: 4.56 s


In [10]:
%%time

xgb_trained_model_cpu = xgboost.train(dtrain=dtrain, params=model_params_cpu, 
                              num_boost_round=model_params_cpu['n_estimators'], verbose_eval=False)

[22:35:04] WARNING: /opt/conda/envs/rapids/conda-bld/xgboost_1612969655237/work/src/learner.cc:541: 
Parameters: { max_features, n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


CPU times: user 21min 26s, sys: 7.27 s, total: 21min 33s
Wall time: 3min 16s


In [11]:
# Make predictions using the trained models, convert the predictions into yes (1.0) / no (0.0) values, and score it against the actual results
predictions = xgb_trained_model.predict(dtest)
predictions = (predictions > threshold) * 1.0   
score = cuml.accuracy_score(y_test, predictions)
print( f'XGBoost accuracy score (GPU) : {score}') 

predictions = xgb_trained_model_cpu.predict(dtest)
predictions = (predictions > threshold) * 1.0   
score = cuml.accuracy_score(y_test, predictions)
print( f'XGBoost accuracy score (CPU) : {score}') 

XGBoost accuracy score (GPU) : 0.9802297353744507
XGBoost accuracy score (CPU) : 0.9805055856704712


#### <font color='#8735fb'> Results

In the previous cell, you should see that both the CPU and GPU models trained to a high accuracy level of around 98 percent. However, the CPU model took around 10 minutes, whereas the GPU model took under 20 seconds. That's a big difference in training time just to get the same accuracy. Now try bumping up the number of estimators in the GPU model and see how that changes performance.

Now that we have the model trained, we save it to a `model` file.

In [12]:
# Save the GPU XGBoost model
xgb_trained_model.save_model(xgb_model_file)

### <font color='#8735fb'> Random Forests
    
Random Forests are another model type. The representation is very similar to XGBoost (an ensemble of decision trees), but the shape, count, and training method of the decision trees are slightly different.
    
For this model, we will be using two different libraries. The CPU model comes from the [sklearn.ensemble.RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) module. The GPU model comes from the RAPIDS [cuml.ensemble.RandomForestClassifier](https://docs.rapids.ai/api/cuml/nightly/api.html?highlight=randomforestclassifier#cuml.ensemble.RandomForestClassifier) module.
    
We will modify the `n_estimators` and `max_depth` hyperparameters, but there are a few more that we could try changing if you refer to the documentation.

In [13]:
# Set the Random Forest hyperparameters
rf_n_estimators = 10
rf_max_depth = 5

In [14]:
%%time

# Use some initial model parameters and the processed training data to train an ML model
rf_model = RandomForestClassifier(n_estimators=rf_n_estimators, max_depth=rf_max_depth)

rf_trained_model = rf_model.fit(X_train.astype('float32'), y_train.astype('int32'))

CPU times: user 3.66 s, sys: 548 ms, total: 4.21 s
Wall time: 3.85 s


In [15]:
%%time

# Use some initial model parameters and the processed training data to train an ML model
rf_model_cpu = RandomForestClassifierCPU(n_estimators=rf_n_estimators, max_depth=rf_max_depth)

rf_trained_model_cpu = rf_model_cpu.fit(X_train_cpu.astype('float32'), y_train_cpu.astype('int32'))

<timed exec>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


CPU times: user 2min 25s, sys: 1.78 s, total: 2min 26s
Wall time: 2min 27s


In [16]:
# Make predictions using the trained models, convert the predictions into yes (1.0) / no (0.0) values, and score it against the actual results
predictions = rf_trained_model.predict(X_test)
predictions = (predictions > threshold) * 1.0   
score = cuml.accuracy_score(y_test, predictions)
print( f'Random Forest accuracy score (GPU) : {score}')

predictions = rf_trained_model_cpu.predict(X_test_cpu)
predictions = (predictions > threshold) * 1.0   
score = cuml.accuracy_score(y_test, predictions)
print( f'Random Forest accuracy score (CPU) : {score}')

Random Forest accuracy score (GPU) : 0.9255858063697815
Random Forest accuracy score (CPU) : 0.931995153427124


#### <font color='#8735fb'> Results
    
In the previous cell, you should see that both the CPU and GPU models trained to a slightly lower level of accuracy than XGBoost, at around 93 percent. However, this time the CPU model only took a few minutes, while the GPU model took around one second, again reaching similar levels of accuracy. Now try bumping up the number of estimators in the GPU model and see how that changes performance. And try it again for the CPU model (if you are patient enough to wait for 30 minutes with `n_estimators` set to 100--and as a hint, this will not improve the accuracy much).
    
Consider the tradeoff here. The overall training time for Random Forests was lower, but at the cost of some accuracy. Again, try playing with some of the hyperparameters and see if you can get the Random Forest accuracy higher than the XGBoost accurucay. Based on the underlying model architectures, this might be a very difficult task!

Now we save the trained model. This time, we serialize it to a `pkl` file, just to see a different way of doing things.

In [17]:
# Save the Random Forest model to for later reloading and re-use, note that we can use external libraries such as pickle to accomplish this
pickle.dump(rf_trained_model, open(rf_model_file, "wb"))

## <font color='#8735fb'> Benchmarks Results

Now that you have worked end to end from data ingest to extract, transform, and load (ETL) to training and some inference, you should have a general feel for how RAPIDS works and what type of speed boosts you can get.
    
In our own setup, we saw some of the numbers and speedup numbers shown in the following screen capture. Hopefully, you saw something similar when you ran through the Notebook. 

<center><a><img src="img/benchmark_times.png" alt="nvidia_logo" style="width: 800px;"/></a></center>

Take a look at the following plot and at how much CPU time is spent in augmentation and training, the two most common operations, versus the GPU workloads. That is a lot of saved time and, as you saw, and it only needed a few small modifications in the code to accomplish them.

<center><a><img src="img/p3_2xlarge_perf.png" alt="nvidia_logo" style="width: 500px;"/></a></center>

# <font color='#8735fb'> Summary

In this Notebook, we trained two CPU models and two GPU models. We then compared the accuracy and training time of all of them.
    
We saw a rather extreme difference in the training times between the CPU and GPU models, with a similar accuracy. We also learned how similar the RAPIDS API was to cpu-models in the process.
    
Hopefully, you had the chance to explore some of the hyperparameters a little more deeply, and saw how deeper models and more trees impacted both performance and training time.
    
By the end of the Notebook, we produced two models and saved them:
* XGBoost: `xgb_trained_model.model`
* Random Forest `rf_trained_model.pkl`
    
These will be used in the next lesson to do further benchmarks and learn about GPU accelerated inferences with the RAPIDS Forest Inference Library backend.

<center><a href="https://developer.nvidia.com/machine-learning"><img src="img/nvidia.png" alt="nvidia_logo" style="width: 1000px;"/></a></center>